In [ ]:
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB7
import tensorflow_hub as hub

import shutil
import os
import random

In [ ]:
def training_graph(history, epochs):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize = (20, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label = 'Training Accuracy')
    plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
    plt.legend(loc = 'lower right')
    plt.title('Training and Validation Accuracy')
    plt.ylabel("Accuracy (training and validation)")
    plt.xlabel("Training Steps")

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label = 'Training Loss')
    plt.plot(epochs_range, val_loss, label = 'Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.ylabel("Loss (training and validation)")
    plt.xlabel("Training Steps")
    plt.show()

In [ ]:
base_path = os.getcwd()
print("Base URL :", base_path)

image_size = (244, 244)
batch_size = 64

train_dir = base_path + '/data/train'
validation_dir = base_path + '/data/validation'

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 90,
    horizontal_flip = True,
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2, 
    zoom_range = 0.5,
    fill_mode = 'nearest')
train_generator = train_datagen.flow_from_directory(
    train_dir,  
    shuffle = True, 
    seed = 42,
    color_mode = "rgb", 
    class_mode = "categorical",
    target_size = image_size,
    batch_size = batch_size)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    shuffle = False, 
    seed = 42,
    color_mode = "rgb", 
    class_mode = "categorical",
    target_size = image_size,
    batch_size = batch_size)

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, log = {}):
        if(log.get('accuracy') > 0.99):
            if(log.get('val_accuracy') > 0.99):
                print("\n Reached 99% Accuracy for both train and val.")
                self.model.stop_training = True

callbacks = MyCallback()

In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1", trainable=False, name="EfficientNetB7"),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(num_classes, activation = 'softmax')
])

model.build([None, image_size[0], image_size[1], 3])
model.summary()

In [ ]:
name_model = "EfficientNetB7_Model"
model.save(base_path + "/model/{}".format(name_model))
#model = tf.keras.models.load_model(base_path + "/model/{}".format(name_model))

In [ ]:
learning_rate = 0.001
epochs = 10

for x in range(7):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate), 
        loss = 'categorical_crossentropy',
        metrics = ['accuracy'])

    history = model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples // train_generator.batch_size,
        epochs = epochs,
        validation_data = validation_generator,
        callbacks = [callbacks],
        validation_steps = validation_generator.samples // validation_generator.batch_size)
    
    model.save(base_path + "/model/{}".format(name_model))

In [ ]:
training_graph(history, epochs)